### Using the model from the experiment that provided the best MSE value
Look up the results in MLFlow for the best experiment and retrieve model name and MLFlow run_id, then fill them in the appropriate cells in this notebook

In [ ]:
%run '../lib/imports.ipynb'

In [ ]:
modelTypeName = "ResNet_valueStdScaled"

In [ ]:
dataColumnName = 'valueStdScaled'

In [ ]:
%run '../lib/utils_anomaly_detection.ipynb'

In [ ]:
%run '../lib/model_commons.ipynb'

In [ ]:
from pathlib import Path
import os, sys

dataFileList = !ls ../../data/rq2-valid/*msg-w-spikes*.csv
dataLatencyFileList =  !ls ../../data/rq2-valid/*avg-latency*.csv

In [ ]:
dataFileList

In [ ]:
pd.set_option('display.max_rows', 5000)
pd.pandas.set_option('display.max_columns', None)

In [ ]:
mlflow_run_id = '<get_the_run_id_from_MLFlow>'

In [ ]:
model_name = '<get_the_model_name_from_MLFlow>'

In [ ]:
mlflow.set_tracking_uri(MLFLOW_URI)
mlflow.set_registry_uri(MLFLOW_URI)

In [ ]:
#model = mlflow.tensorflow.load_model(f'models:/{model_name}/1')

model_params = mlflow.get_run(mlflow_run_id).to_dictionary()['data']['params']

In [ ]:
model_params

In [ ]:
T = int(model_params['time_window'])

In [ ]:
####
# Set the target environment to cluster or local. Default is local
####
run_local = True

In [ ]:
if run_local == False:
    print("Running on Cluster")
    from codeflare_sdk import TokenAuthentication
    from codeflare_sdk import generate_cert
    
    auth = TokenAuthentication(
        token = "<your_access_token>", # execute ocp whoami -t on the authenticated cluster to obtain the token
        server = "<OCP cluster API URL>",
        skip_tls = True
    )
    auth.login()
    
    # Create required TLS cert and export the environment variables to enable TLS
    generate_cert.generate_tls_cert('raycluster-complete', 'raycluster')
    generate_cert.export_env('raycluster-complete', 'raycluster')
    
    ray_endpoint = 'ray://raycluster-complete-head-svc.raycluster.svc.cluster.local:10001' # ensure your ray cluster URL is correct
    ray.shutdown()
    ray.init(address=ray_endpoint, logging_level=logging.ERROR, log_to_driver=False)
else:
    ray.shutdown()
    ray.init()    

In [ ]:
results_trials = {}
for filePos in range(len(dataFileList)):
    
    data = pd.read_csv(dataFileList[filePos], index_col='EventDateTime', parse_dates=['EventDateTime'])
    dataLatency = pd.read_csv(dataLatencyFileList[filePos], index_col='EventDateTime', parse_dates=['EventDateTime'])
    print(f'Processing files at position {filePos} in list')
    %run '../lib/prepareDataSet.ipynb'

    trial_fname = os.path.basename(dataFileList[filePos])
    results_predict_ahead = {}
    for predict_ahead in [5, 10, 15, 30, 45, 60, 90, 120]:
        res = run_new_data_evaluation.remote(model_name, mlflow_run_id, T, predict_ahead, trial_fname)
        tag = f'predict_ahead_{predict_ahead}-trial_{trial_fname}'
        results_predict_ahead[tag] = res
        
    results_trials[trial_fname] = results_predict_ahead    

In [ ]:
results_trials

In [ ]:
data_trials = {}
for trial in results_trials.keys():
    data_trial = {}
    for item in results_trials[trial].keys():
        try:
            res = ray.get(results_trials[trial][item])
        except:
            print(f'Error getting results for key:{item}')
            res = None
        data_trial[item] = res
        
    data_trials[trial] = data_trial

In [ ]:
data_trials

In [ ]:
ray.shutdown()

In [ ]:
def find_best_result():
    min_mse = 1000
    tag = ''
    result = {}
    for trial in data_trials.keys():
        crt_trial = data_trials[trial]
        for predict_ahead in crt_trial.keys():
            crt_one_step = crt_trial[predict_ahead]['one_step_metrics']
            if crt_one_step['mse_1Step'] < min_mse:
                min_mse = crt_one_step['mse_1Step']
                tag = str(predict_ahead) + '-' + str(trial)
                result = crt_trial[predict_ahead]

    return result, tag
                

In [ ]:
find_best_result()